In [ ]:
from astropy.io.fits import getdata, getheader
import astropy.units as u
from nustar_lunar_pointing.tracking import get_epoch_tle
from nustar_lunar_pointing.tracking import convert_nustar_time
from nustar_lunar_pointing.tracking import get_moon_j2000
from astropy.time import Time


import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
from datetime import datetime

checktime = datetime.strptime('2017-05-08', "%Y-%m-%d")
tlefile = '../data/NuSTAR.tle'
mindt, line1, line2 = get_epoch_tle(checktime, tlefile)
print('Days between TLE entry and when you want to observe: ', mindt)


In [ ]:
from skyfield.api import EarthSatellite, load

ts = load.timescale()

planets = load('de436.bsp')
moon = planets['Moon']
earth = planets['Earth']


checktime = datetime.strptime('2017-05-08', "%Y-%m-%d")
tlefile = '../data/NuSTAR.tle'
mindt, line1, line2 = get_epoch_tle(checktime, tlefile)
print('Days between TLE entry and when you want to observe: ', mindt)
nustar = EarthSatellite(line1, line2)
geometry = nustar +earth



In [ ]:
from astropy.coordinates import SkyCoord
from datetime import timedelta
ra = []
dec =[]
times= []
base_ra = None
base_dec = None


ra_sky = []
dec_sky = []

step_size = timedelta(0, 1000.) # 1000 second steps
checktime = datetime.strptime('2017-05-08', "%Y-%m-%d")
end_check = datetime.strptime('2017-05-10', "%Y-%m-%d")


while ( (end_check - checktime).total_seconds() ) > 0:
    checktime += step_size
        
    ra_moon, dec_moon = get_moon_j2000(checktime, line1, line2)
    utc = Time(checktime)
    tcheck = ts.from_astropy(utc)

    times.extend([checktime])
    ra.extend([ra_moon.value])
    dec.extend([dec_moon.value])
 
    nustar_bary = geometry.at(tcheck)
    astrometric = nustar_bary.observe(moon)
    this_ra_sky, this_dec_sky, dist = astrometric.radec()

    ra_sky.extend([this_ra_sky.to(u.deg).value])
    dec_sky.extend([this_dec_sky.to(u.deg).value])

    astropy_coord = SkyCoord(ra_moon, dec_moon)
    sky_coord = SkyCoord(this_ra_sky.to(u.deg), this_dec_sky.to(u.deg))
    print(astropy_coord.separation(sky_coord))
    

In [ ]:
ax = plt.figure(112, figsize=(8, 6))
plt.plot(times, ra, 'g.')
plt.plot(times, ra_sky, 'b.')
plt.xlabel('Time (sec)')
plt.ylabel('RA (deg)')

ax = plt.figure(212, figsize=(8, 6))


plt.plot(times, dec, 'b.')
plt.xlabel('Time (sec)')
plt.ylabel('Dec (deg)')

